# **Assignment 3**
## Yingjie Chen (yc24966) - 08:30 section
## Andrew Han (ah49372) - 10:30 section
## Sitong Li (sl43736) - 10:30 section
## Matthew Ruffner (mar9632) - 08:30 section
## Xinmeng Song (xs3659) - 08:30 section

#Scraping:

In [ ]:
!pip install --user google-cloud
!pip install --user google-cloud-vision

In [ ]:
!pip install instaloader

     |████████████████████████████████| 51kB 1.5MB/s 
  Created wheel for instaloader: filename=instaloader-4.5.3-cp36-none-any.whl size=50844 sha256=05d1459ebb43475156afacc63cbab5a230b3a408e4107cc32f2e70f944eab641
  Stored in directory: /root/.cache/pip/wheels/3f/5a/0e/bf22dafc60d940c4c30734cb6ed2bb5bbca2784a77961f77da
Successfully built instaloader


In [ ]:
!pip install datetime
#!pip install itertools

     |████████████████████████████████| 61kB 1.7MB/s 
     |████████████████████████████████| 245kB 7.2MB/s 


In [ ]:
from google.cloud import vision
import xlrd
import os
import pandas as pd
import instaloader
import time
import sys
sys.setrecursionlimit(10000)
from datetime import datetime 
from itertools import dropwhile, takewhile
from pandas import Series, DataFrame
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

(i) image URLs (do not extract video URLs, it may end up costing you a lot of money to run analytics on video)
(ii) post caption (the text description of a post)
(iii) # likes 
(iv) # comments. You don’t need actual comments for this assignment. 
Scrape around 500 image posts. 

In [ ]:
L = instaloader.Instaloader()
df = pd.DataFrame()
i = 0
for post in instaloader.Profile.from_username(L.context, 'zara').get_posts():
    
    if post.typename == 'GraphImage': #exclude videos and GraphSidecar (multiple pics)
      df = df.append({'URL':post.url, 'Caption':post.caption, 'Likes': post.likes, 'Comments': post.comments }, ignore_index=True)
      df.to_excel("zara.xlsx",index = False)
    else:
      i = i-1
    i = i+1
    if i>499:
        break

Using the image URLs, obtain image labels from Google Vision cloud (you will have to create an account with Google to get your credentials as a json file, though the first $300 are free, which should be more than plenty for this assignment). You will need to write a script to access the Google Vision API.  Read about Google Vision here: For Google Vision API, look here: https://cloud.google.com/vision/docs/quickstart

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving zara-ins-hw3-6b2d402b5998.json to zara-ins-hw3-6b2d402b5998.json


In [ ]:
#google cloud script
Application_Credentials = 'zara-ins-hw3-6b2d402b5998.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision.Image()


In [ ]:
loc = ("zara.xlsx")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0,3)

'URL'

In [ ]:
df = pd.DataFrame()
#loop through url, retreive and send to google vision
for i in range(sheet.nrows):
    image_src_temp = sheet.cell_value(i,3)
    image.source.image_uri = image_src_temp
    response = client.label_detection(image=image)
    labels = response.label_annotations
    l = []
    for label in labels:
      l.append(label.description)
    s = ' '.join(l)
    print("s")
    print(s)
    df = df.append({'URL': image_src_temp, 'Labels': s}, ignore_index = True)
df.to_excel("Labels.xlsx",index=False)

s

s
Sky Cool Architecture Photography Vacation Tourism Travel Smile Cloud House
s
Jacket Photography Top Sleeve
s
Blue Turquoise Cool Musician Musical instrument Electric blue Street fashion Street performance
s
Cool Human Font Fur Outerwear Textile Album cover Furniture Fashion accessory Black hair
s
Photograph Clothing Formal wear Standing Suit Fashion Coat Overcoat Black-and-white Outerwear
s
Suit Fashion Formal wear Outerwear Fashion design
s
Clothing Standing Outerwear Coat Fashion Overcoat Formal wear Blazer Black-and-white Suit
s
Face Hair Eyebrow Forehead Hairstyle Chin Lip Skin Blond Beauty
s
Face Sitting Beauty Black-and-white Model Photo shoot Photography Monochrome Monochrome photography Style
s
Hair Clothing Street fashion Fashion Beauty Coat Hairstyle Outerwear Snapshot Fashion model
s
Clothing Standing Dress Shoulder Black-and-white Fashion Joint Costume Footwear Outerwear
s
Hair Beauty Skin Photo shoot Fashion model Model Long hair Hairstyle Shoulder Fashion
s
Sitting 

#Task A:

In [ ]:
data_xls = pd.read_excel('zara.xlsx', index_col=None)
data_xls.to_csv('zara.csv', encoding='utf-8', index=False)
df = pd.read_csv('zara.csv')

max_likes = df['Likes'].max()
max_comments = df['Comments'].max()
df['scaled_numlikes']=df['Likes']/max_likes
df['scaled_numcomments']=df['Comments']/max_comments
df['engagement_score']=.4*df['scaled_numlikes']+.6*df['scaled_numcomments']

median_escore=np.median(df['engagement_score'])

df['high/low_engagement']=1
for i in range (len(df)):
    if df['engagement_score'][i] < median_escore:
        df['high/low_engagement'][i] = 0
    else:
        df['high/low_engagement'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Task B:


In [ ]:
labels = pd.read_excel("Labels.xlsx")
labels.drop(0, axis=0, inplace=True)
labels

,Labels,URL
1,Sky Cool Architecture Photography Vacation Tou...,https://scontent-iad3-1.cdninstagram.com/v/t51...
2,Jacket Photography Top Sleeve,https://scontent-iad3-1.cdninstagram.com/v/t51...
3,Blue Turquoise Cool Musician Musical instrumen...,https://scontent-iad3-1.cdninstagram.com/v/t51...
4,Cool Human Font Fur Outerwear Textile Album co...,https://scontent-iad3-1.cdninstagram.com/v/t51...
5,Photograph Clothing Formal wear Standing Suit ...,https://scontent-iad3-1.cdninstagram.com/v/t51...
...,...,...
496,Vehicle Vehicle door Car Motor vehicle Driving...,https://scontent-iad3-1.cdninstagram.com/v/t51...
497,Photograph Yellow Snapshot Turquoise T-shirt C...,https://scontent-iad3-1.cdninstagram.com/v/t51...
498,Photograph Vehicle Snapshot Car Automotive ext...,https://scontent-iad3-1.cdninstagram.com/v/t51...
499,Photograph Black Standing Fashion Model Cool O...,https://scontent-iad3-1.cdninstagram.com/v/t51...


Label

In [ ]:
labels["Labels"] = labels["Labels"].apply(lambda x: str(x).lower())
corpus = labels["Labels"].tolist()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names()
# print(word)
# print (X.toarray())
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
# print(transformer)
tfidf = transformer.fit_transform(X)
# print (tfidf.toarray())

df2 = pd.DataFrame.sparse.from_spmatrix(tfidf)
renames = {i:j for i,j in zip(range(539),word)}
df2.rename(columns=renames, inplace=True)
df2["engagement"] = df["high/low_engagement"]

from patsy import dmatrices
%pylab inline
formula = 'engagement ~ 0 + Q(\'' + '\') + Q(\''.join(word) + '\')'
Y, X = dmatrices(formula, df2, return_type='dataframe')
y = Y['engagement'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)
from sklearn import metrics

prediction_train = model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train))

prediction = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))

Populating the interactive namespace from numpy and matplotlib
0.7885714285714286
0.52


Caption

In [ ]:

df["lower_cap"] = df["Caption"].apply(lambda x: str(x).lower())
corpus = df["lower_cap"].tolist()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names()
word = word[44:]
# print(len(word))
# print (X.toarray())
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
# print(transformer)
tfidf = transformer.fit_transform(X)
# print (tfidf.toarray())

df3 = pd.DataFrame.sparse.from_spmatrix(tfidf)
df3.drop(df3.iloc[:, 0:44], inplace=True, axis=1)
renames = {i:j for i,j in zip(range(44, 1342),word)}
df3.rename(columns=renames, inplace=True)
df3["engagement"] = df["high/low_engagement"]
formula = 'engagement ~ 0 + Q(\'' + '\') + Q(\''.join(word) + '\')'

Y, X = dmatrices(formula, df3, return_type='dataframe')
y = Y['engagement'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)
from sklearn import metrics

prediction_train = model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train))
prediction = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))


0.8971428571428571
0.68


Conclusion:
We got a higher accuracy score in predicting the engagement scores when regressed relative to caption keywords instead of labels. Therefore, to increase engagement on instagram, Zara should focus on writing out the caption instead of relying on the image labels to write out the captions. The company could predict the caption more accurately when it does the further analysis on next season.

# Task C

In [ ]:
df = pd.read_excel('zara.xlsx', index_col = None)
df.to_csv('zara.csv')
df= pd.read_csv('zara.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.drop([65])
df = df.reset_index()
df = df.drop(['index'], axis = 1)

# scale the likes
max_likes = df['Likes'].max()
max_comments = df['Comments'].max()

df['scaled_numlikes']=df['Likes']/max_likes
df['scaled_numcomments']=df['Comments']/max_comments

df['engagement_score']=.4*df['scaled_numlikes']+.6*df['scaled_numcomments']
df['engagement_score']

median_escore=np.median(df['engagement_score'])
median_escore

df['high/low_engagement']=1
for i in range (len(df)):
    if df['engagement_score'][i] < median_escore:
        df['high/low_engagement'][i] = 0
    else:
        df['high/low_engagement'][i] = 1

labels = pd.read_excel('Labels.xlsx', index_col = None)
labels.to_csv('labels.csv')
labels = pd.read_csv('labels.csv', dtype=str)
labels = labels.drop(['Unnamed: 0'], axis = 1)
labels = labels.drop([0])
labels.iloc[64]
labels = labels.dropna()
labels = labels.reset_index()
labels = labels.drop(['index'], axis = 1)
labels

import re
# from nltk.corpus import stopwords
# nltk.download('stopwords')
labels["Labels"] = labels["Labels"].str.lower()
def vectors(string):
    return string.split()
labels["Labels"] = labels["Labels"].map(vectors)

list_of_list_of_tokens=[]
for i in range(len(labels)):
    list_of_list_of_tokens.append(labels['Labels'][i])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Don't run it again!
from gensim import corpora, models
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 4
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()



/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 1.4 s, sys: 17.2 ms, total: 1.41 s
Wall time: 1.39 s
0: 0.051*"blue" + 0.039*"photography" + 0.039*"shoulder" + 0.034*"sitting" + 0.031*"white" + 0.030*"jeans" + 0.030*"fashion" + 0.030*"denim" + 0.026*"leg" + 0.026*"beauty"

1: 0.077*"photography" + 0.068*"hair" + 0.034*"beauty" + 0.026*"hairstyle" + 0.025*"child" + 0.024*"photo" + 0.024*"shoot" + 0.023*"skin" + 0.022*"model" + 0.022*"sitting"

2: 0.071*"fashion" + 0.051*"clothing" + 0.048*"outerwear" + 0.034*"neck" + 0.033*"sleeve" + 0.030*"shoulder" + 0.025*"jacket" + 0.021*"blue" + 0.020*"cool" + 0.019*"shoe"

3: 0.126*"fashion" + 0.068*"clothing" + 0.054*"outerwear" + 0.039*"model" + 0.039*"dress" + 0.034*"shoulder" + 0.029*"leg" + 0.027*"standing" + 0.023*"white" + 0.023*"beauty"



Topic 0: DenimClothing

Topic 1: Photography

Topic 2: WinterClothing 

Topic 3: SummerClothing

In [ ]:
#printing out the % of topics each image is about (topic, weight)
topic_list=[]
for i in range(len(corpus)):
    topic_list.append(lda_model[corpus[i]])
    #print(lda_model[corpus[i]])

dic = []
for i in range(len(topic_list)):
    dic.append(dict(topic_list[i]))

df1 = pd.DataFrame(dic)
df1.fillna(0)

,1,2,3,0
0,0.996681,0.000000,0.000000,0.000000
1,0.000000,0.992574,0.000000,0.000000
2,0.000000,0.997012,0.000000,0.000000
3,0.000000,0.997283,0.000000,0.000000
4,0.000000,0.000000,0.997283,0.000000
...,...,...,...,...
494,0.000000,0.000000,0.997283,0.000000
495,0.653828,0.344180,0.000000,0.000000
496,0.000000,0.000000,0.997699,0.000000
497,0.000000,0.000000,0.997012,0.000000


In [ ]:
labels = labels.join(df1)
labels = labels.fillna(0)
labels = labels.rename(columns={0: "Topic0", 2: "Topic2", 1:"Topic1", 3:"Topic3"})
labels

,Labels,URL,Topic1,Topic2,Topic3,Topic0
0,"[sky, cool, architecture, photography, vacatio...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.996681,0.000000,0.000000,0.000000
1,"[jacket, photography, top, sleeve]",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.992574,0.000000,0.000000
2,"[blue, turquoise, cool, musician, musical, ins...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.997012,0.000000,0.000000
3,"[cool, human, font, fur, outerwear, textile, a...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.997283,0.000000,0.000000
4,"[photograph, clothing, formal, wear, standing,...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.000000,0.997283,0.000000
...,...,...,...,...,...,...
494,"[vehicle, vehicle, door, car, motor, vehicle, ...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.000000,0.997283,0.000000
495,"[photograph, yellow, snapshot, turquoise, t-sh...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.653828,0.344180,0.000000,0.000000
496,"[photograph, vehicle, snapshot, car, automotiv...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.000000,0.997699,0.000000
497,"[photograph, black, standing, fashion, model, ...",https://scontent-iad3-1.cdninstagram.com/v/t51...,0.000000,0.000000,0.997012,0.000000


In [ ]:
final_df=df.merge(labels, on='URL')
#final_df.head()

# max_score = max(final_df['engagement_score'])
# min_score = min(final_df['engagement_score'])

final_df['quartiles']=pd.qcut(final_df['engagement_score'], 4)
final_df=final_df.rename(columns={"Topic0": "DenimClothing", "Topic1":"Photography", "Topic2":"WinterClothing", "Topic3":"SummerClothing"})
final_df.head()


,Caption,Comments,Likes,URL,scaled_numlikes,scaled_numcomments,engagement_score,high/low_engagement,Labels,Photography,WinterClothing,SummerClothing,DenimClothing,quartiles
0,ZARA / CITIES. A project shot in the streets o...,36,15171,https://scontent-iad3-1.cdninstagram.com/v/t51...,0.051761,0.011091,0.027359,0,"[sky, cool, architecture, photography, vacatio...",0.996681,0.000000,0.000000,0.0,"(0.0264, 0.105]"
1,ZARA / CITIES. A project shot in the streets o...,88,22328,https://scontent-iad3-1.cdninstagram.com/v/t51...,0.076179,0.027110,0.046738,0,"[jacket, photography, top, sleeve]",0.000000,0.992574,0.000000,0.0,"(0.0264, 0.105]"
2,ZARA / CITIES. A project shot in the streets o...,82,30398,https://scontent-iad3-1.cdninstagram.com/v/t51...,0.103713,0.025262,0.056642,0,"[blue, turquoise, cool, musician, musical, ins...",0.000000,0.997012,0.000000,0.0,"(0.0264, 0.105]"
3,FW20 campaign. Coming soon #zaracampaign #zara...,270,36039,https://scontent-iad3-1.cdninstagram.com/v/t51...,0.122959,0.083179,0.099091,0,"[cool, human, font, fur, outerwear, textile, a...",0.000000,0.997283,0.000000,0.0,"(0.0264, 0.105]"
4,FW20 campaign. Coming soon #zaracampaign #zara...,199,38601,https://scontent-iad3-1.cdninstagram.com/v/t51...,0.131700,0.061306,0.089464,0,"[photograph, clothing, formal, wear, standing,...",0.000000,0.000000,0.997283,0.0,"(0.0264, 0.105]"


In [ ]:
final_df[["quartiles", "DenimClothing", "Photography", "WinterClothing", "SummerClothing", "engagement_score"]].groupby("quartiles")[["DenimClothing", "Photography", "WinterClothing", "SummerClothing"]].mean()

,DenimClothing,Photography,WinterClothing,SummerClothing
quartiles,,,,
"(0.0264, 0.105]",0.173701,0.340291,0.185203,0.297420
"(0.105, 0.141]",0.168837,0.291261,0.185496,0.351332
"(0.141, 0.204]",0.123682,0.319388,0.166279,0.387324
"(0.204, 0.716]",0.111559,0.284365,0.194713,0.406527


Conclusion:
We got four topics which are: ‘DenimClothing’, ‘Photography’, ‘WinterClothing’ and ‘SummerClothing’. The word ‘fashion’ appears in all of these categories, which is similar to Zara’s brand positioning. The price of zara clothing is relatively flat and reasonable. It’s target audience consists mainly of young people aged 20 to 35. Customers in this age group are characterized by their consumption level. At the middle level of society, but also pursuing fashionable and novel styles. And the increasing fashion characteristics of different products(jean clothing, summer clothing and winter clothing) can improve the increasing engagement. As for  photography which is related to Zara product photos, it shows that the selection of models and the styling of models taken by the models can increase engagement. 

# Task D



A major discovery our analysis has shown is that instagram users tend to engage more with the current season’s style instead of out of season style. Having scraped instagram from early fall we saw higher engagement on outerwear associated with warmer weather than colder weather fashion. Additionally, we noticed a negative engagement trend in denim clothing indicating denim fashion may be going out of style. In conclusion, we would recommend that Zara move away from denim clothing and focus on the current season’s clothing style instead of trying to advertise for the upcoming seasons. Going forward, since Zara seems to be able to predict engagement more accurately using the captions on their photos, when planning how they will be marketing their products for future seasons they are better able understand their audience’s sentiment towards a certain product by using said product in a hashtag that users can easily search for on the instagram.